In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import datetime 
import warnings
import csv
warnings.simplefilter(action='ignore', category=FutureWarning)
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [2]:
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
item_cat = pd.read_csv ('../input/competitive-data-science-predict-future-sales/item_categories.csv')
trainset1 = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')
testset= pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')

In [3]:
trainset1.info()

In [4]:
items.info()

In [5]:
item_cat.info()

In [6]:
shops.info()

In [7]:
trainset1.shape

In [8]:
display(trainset1, item_cat, items, shops)

In [9]:
trainset2=pd.merge(items, trainset1)

In [10]:
trainset3=pd.merge(item_cat, trainset2)

In [11]:
train=pd.merge(shops, trainset3)

In [12]:
train.info()

In [13]:
train.shape

In [14]:
train.describe()

In [15]:
train.isnull().sum()

In [16]:
train['shop_name'].unique()

In [17]:
train['shop_id'].unique()

In [18]:
train['item_category_name'].unique()

In [19]:
train['item_name'].unique()

In [20]:
train['item_id'].unique()

In [21]:
train['shop_name'].nunique()

In [22]:
train['shop_id'].nunique()

In [23]:
train['item_category_name'].nunique()

In [24]:
train['item_name'].nunique()

In [25]:
train['item_id'].nunique()

In [26]:
train_set=train.copy()

In [27]:
corrmat=train_set.corr()
corrmat

In [28]:
plt.figure(figsize=(15, 15))
sns.set(font_scale=2)
sns.heatmap(train_set.corr(), annot=True, cmap="Blues", fmt= ".2g")

In [29]:
negative_price=train_set[train_set["item_price"] < 0]
negative_price

In [30]:
price_2973=train_set[(train_set["shop_id"]== 32) & (train_set["item_category_name"]=="Игры - PS3") & (train_set["item_id"]== 2973 ) & (train_set["date_block_num"]== 4)]
price_2973

In [31]:
price_mean=price_2973[(price_2973.shop_id==32)&(price_2973.item_id==2973)&(price_2973.date_block_num==4)&(price_2973.item_price>0)]
price_mean.item_price.mean()

In [32]:
train_set.at[1644072,'item_price']= 1849

In [33]:
negative_item_cnt_day=train_set[train_set["item_cnt_day"] < 0]
negative_item_cnt_day

In [34]:
cond = train_set['item_cnt_day'] < 0
check= train_set.loc[cond,'item_cnt_day'] = 0
check

In [35]:
plt.figure(figsize=(10, 15))
train_set.boxplot(['item_price'])
plt.title('Outliers in item_price', fontsize = 40)

In [36]:
print("Maximum possible price",train_set['item_price'].mean() + 3*train_set['item_price'].std())
print("Lowest possible price",train_set['item_price'].mean() - 3*train_set['item_price'].std())

In [37]:
hp=max(train_set.item_price)
print("Highest price :", hp)
lp=min(train_set.item_price)
print("Lowest price: ", lp)

In [38]:
highest_price=train_set[train_set["item_price"] > 6080]
highest_price

In [39]:
train_set = train_set[train_set['item_price'] < 100000]

In [40]:
plt.figure(figsize=(10, 15))
train_set.boxplot(['item_cnt_day'])
plt.title('Outliers in item_cnt_day', fontsize = 40)

In [41]:
print("Maximum possible number of products sold",train_set['item_cnt_day'].mean() + 3*train_set['item_cnt_day'].std())
print("Lowest possible number of products sold",train_set['item_cnt_day'].mean() - 3*train_set['item_cnt_day'].std())

In [42]:
hps=max(train_set.item_cnt_day)
print("Highest number of products sold :", hps)
lps=min(train_set.item_cnt_day)
print("Lowest number of products sold: ", lps)

In [43]:
highest_price=train_set[train_set["item_cnt_day"] > 9]
highest_price

In [44]:
train_set = train_set[train_set['item_cnt_day'] < 1000]

In [45]:
train_set['Sales'] = (train_set["item_price"] * train_set["item_cnt_day"])

In [46]:
train_set['date'] = pd.to_datetime(train_set['date'])
days = []
months = []
years = []

for day in train_set['date']:
    days.append(day.day)
for month in train_set['date']:
    months.append(month.month)    
for year in train_set['date']:
    years.append(year.year)

In [47]:
plt.figure(figsize=(10,10))
sns.countplot(years)
plt.title('Sales per Year', fontsize = 40)
plt.xlabel('Year', fontsize = 25)
plt.ylabel('Sales', fontsize = 25)
plt.show()

In [48]:
plt.figure(figsize=(40, 50))
x= train_set['item_category_id']
y= train_set['Sales']
sns.barplot(x, y)
plt.title('Highest Selling Product', fontsize = 60)
plt.xlabel('Item categories', fontsize = 30)
plt.ylabel('Sales', fontsize = 30)
plt.show()

In [49]:
highest_selling_itemc= train_set.loc[train_set['item_category_id'] == 12]
highest_selling_itemc

In [50]:
plt.figure(figsize=(40, 50))
x= train_set['shop_id']
y= train_set['Sales']
sns.barplot(x, y)
plt.title('Highest Selling Shop', fontsize = 60)
plt.xlabel('Shop_id', fontsize = 30)
plt.ylabel('Sales', fontsize = 30)
plt.show()

In [51]:
highest_selling_shop= train_set.loc[train_set['shop_id'] == 9]
highest_selling_shop

In [52]:
plt.figure(figsize=(10,10))
sns.countplot(months)
plt.title('Sales per Year', fontsize = 40)
plt.xlabel('Year', fontsize = 25)
plt.ylabel('Sales', fontsize = 25)
plt.show()

In [53]:
sales_by_month = train_set.groupby(['date_block_num'])[["Sales"]].sum()
plt.figure(figsize=(30,30))
sales_by_month.plot()
plt.title('Month with highest Sales in the dataset', fontsize = 10)
plt.xlabel('Months', fontsize = 10)
plt.ylabel('Sales', fontsize = 10)

In [54]:
sales_max = train_set.groupby(['date_block_num'])['Sales'].sum()
sales_max.max()

In [55]:
print(sales_max==232615420.90999845)

In [56]:
highest_sales_month = train_set[train_set["date_block_num"]==23]
highest_sales_month

In [57]:
highest_store_sales = train_set.groupby(['date_block_num', 'shop_name'])['Sales'].sum()

highest_store_sales.max()

In [58]:
highest_store_sales=pd.DataFrame(highest_store_sales)
highest_store_sales=highest_store_sales.loc[highest_store_sales['Sales'] == 15730394.000000501]
highest_store_sales

In [59]:
highest_item_sales = train_set.groupby(['date_block_num', 'item_category_name'])['Sales'].sum()

highest_item_sales.max()

In [60]:
highest_item_sales=pd.DataFrame(highest_item_sales)
highest_item_sales=highest_item_sales.loc[highest_item_sales['Sales'] == 46487721.9999976]
highest_item_sales

In [61]:
train_set = train_set[["date_block_num", "shop_id", "item_id", "item_price", "item_cnt_day", "shop_name", "Sales", "item_name"]].groupby(
    ["date_block_num", "shop_id", "item_id"]).agg(
    {"item_cnt_day": "sum", "Sales" : "sum"}).reset_index()
train_set.rename(columns={"item_cnt_day": "item_cnt_month"}, inplace=True)

In [62]:
test1=pd.merge(items, testset)
test2=pd.merge(item_cat, test1)
test = pd.merge(shops,test2)
test

In [63]:
test=test.drop("ID",1)

In [64]:
test.shape

In [65]:
train_set_u = train_set['item_id'].nunique()
test_u = test['item_id'].nunique()
train_set_s = train_set['shop_id'].nunique()
test_s = test['shop_id'].nunique()
print("Total unique item_ids in train_set dataset: ", train_set_u)
print("Total unique item_ids in test dataset: ", test_u )

print("Total unique shop_ids train_set dataset: ", train_set_s)
print("Total unique shop_ids in test dataset: ", test_s)

In [66]:
train_set = train_set[train_set['shop_id'].isin(test['shop_id'].unique())]
train_set = train_set[train_set['item_id'].isin(test['item_id'].unique())]

In [67]:
train_set_u = train_set['item_id'].nunique()
test_u = test['item_id'].nunique()
train_set_s = train_set['shop_id'].nunique()
test_s = test['shop_id'].nunique()
print("Total unique item_ids in train_set dataset: ", train_set_u)
print("Total unique item_ids in test dataset: ", test_u )

print("Total unique shop_ids train_set dataset: ", train_set_s)
print("Total unique shop_ids in test dataset: ", test_s)

In [68]:
test.shape

In [69]:
test["item_cnt_month"]=" "

In [70]:
test.info()
train_set.info()

In [71]:
train_set=train_set.drop("Sales", 1)
train_set=train_set.drop("date_block_num", 1)
test= test.drop("item_category_id", 1)
test= test.drop("item_category_name", 1)
test= test.drop("item_name", 1)
test= test.drop("shop_name", 1)

In [72]:
train_x = train_set.drop(['item_cnt_month'], axis=1)
train_y = train_set['item_cnt_month']
test_x = test.drop(['item_cnt_month'], axis=1)
model_lgb = LGBMRegressor(colsample_bytree=0.9, learning_rate=0.03, max_depth=6,
              min_child_weight=1, min_split_gain=0.0222415, n_estimators=100,
              num_leaves=32, reg_alpha=0.04, reg_lambda=0.073,
              subsample=0.9)
model_lgb.fit(train_x, train_y)

In [73]:
sales_prediction = model_lgb.predict(test_x)
sales_prediction

In [75]:
sales_predict_submission = pd.DataFrame({'ID':test_x.index,'item_cnt_month':sales_prediction})

sales_predict_submission

In [76]:
 sales_predict_submission.to_csv('./submission.csv', index= False, header= 1)